In [ ]:
import numpy as np
import pandas as pd
from time import process_time_ns

from nfpy.Assets import get_af_glob, get_fx_glob
import nfpy.Calendar as Cal
from nfpy.Financial import DividendFactory

#### Define functions and prepare data

In [ ]:
def div_paid_numpy(ptf, af, fx):
    # Create the list of years
    years = np.unique(
        self._cal
            .calendar
            .values
            .astype('datetime64[Y]')
    )
    dividends = np.zeros(years.shape[0])

    for uid in ptf.constituents_uids:
        if fx.is_ccy(uid):
            continue

        asset = af.get(uid)

        if asset.type == 'Currency':
            continue

        elif asset.type == 'Equity':
            dt, div = DividendFactory(asset).dividends
            y_dt = dt.astype('datetime64[Y]')

            for n, y in enumerate(years):
                dividends[n] += np.sum(div[y_dt == y])

    return dividends

def div_paid_pandas(ptf, af, fx):
    # Create the list of years
    dividends = pd.Series(.0, index=self._cal.calendar)

    for uid in ptf.constituents_uids:
        if fx.is_ccy(uid):
            continue

        asset = af.get(uid)

        if asset.type == 'Currency':
            continue

        elif asset.type == 'Equity':
            dt, div = DividendFactory(asset).dividends
            divs = pd.Series(div, index=dt) \
                .asfreq(freq='B') \
                .fillna(.0)
            dividends = dividends.add(divs, fill_value=.0)

    return dividends.resample('Y').sum()

In [ ]:
cal = Cal.get_calendar_glob()
cal.initialize(Cal.today(), '2010-01-01')
af = get_af_glob()
fx = get_fx_glob()

In [ ]:
ptf = af.get('IBKR')

for uid in ptf.constituents_uids:
    if fx.is_ccy(uid):
        continue
    _ = af.get(uid)

#### Comparison

In [ ]:
%%timeit
for i in range(1000):
    d_n = div_paid_numpy()

In [ ]:
%%timeit
for i in range(1000):
    d_p = div_paid_pandas()